## Measuring model ddG accuracy on protein variants
The goal of this notebook is to experiment with model ddG prediction capabilities, and exploring the possibility of predicting ddG of variants that were not supported previously.

In [3]:
#imports
import sys
sys.path.append('..')    # add parent directory to path
import numpy as np
import matplotlib.pyplot as plt
from model.hydro_net import PEM
from model.model_cfg import CFG
from Utils.train_utils import *
from Utils.pdb_parser import get_pdb_data
import torch
import pandas as pd
import os
import glob

### Experimenting with a single protein

In [4]:
#Configuration
remove_indels = True
one_mute = True
# Constants
COORDS = 'coords_tensor.pt'
DELTA_G = 'deltaG.pt'
MASKS = 'mask_tensor.pt'
ONE_HOT = 'one_hot_encodings.pt'
PROTT5_EMBEDDINGS = 'prott5_embeddings'
VAL_RATIO = 0.2
RANDOM_SEED = 42
NANO_TO_ANGSTROM = 0.1
TM_PATH = './data/Processed_K50_dG_datasets/TM_proteins.csv'
DEBUG = False

In [5]:
#function from AllProteinValidationDataset
def load_embedding_tensor(self, embeddings_dir):
        embeddings = []
        all_embedding_files = sorted(glob.glob(os.path.join(embeddings_dir, 'prott5_embedding_*.pt')),
                                     key=lambda x: int(os.path.splitext(x)[0].split('_')[-1]))
        for filename in all_embedding_files:
            if filename.endswith('.pt'):
                embedding_tensor = torch.load(filename)
                embeddings.append(embedding_tensor)
        return torch.vstack(embeddings)

In [ ]:
#Data preparation
protein_path = r"Mutation-fineTuning\mutation_data\1A0N"
mutations_path = r"Mutation-fineTuning\mutation_data\1A0N.csv"

if not os.path.exists(mutations_path):
    raise FileNotFoundError(f"File not found: {mutations_path}")

mutations = pd.read_csv(mutations_path)

if remove_indels:
    mutations = mutations[~mutations['mut_type'].str.contains('ins|del')].reset_index(drop=True)
    
# Load and preprocess the data for each protein
coords_tensor = torch.load(os.path.join(protein_path, COORDS))
delta_g_tensor = torch.load(os.path.join(protein_path, DELTA_G))
mask_tensor = torch.load(os.path.join(protein_path, MASKS))
one_hot_tensor = torch.load(os.path.join(protein_path, ONE_HOT))
embedding_tensor = load_embedding_tensor(os.path.join(protein_path, PROTT5_EMBEDDINGS))

# remove the mutations with more than one mutation
if one_mute:
    one_mut_index = mutations[~mutations['mut_type'].str.contains(':')]
    mutations = mutations.loc[one_mut_index.index]
    delta_g_tensor = delta_g_tensor[one_mut_index.index]
    one_hot_tensor = one_hot_tensor[one_mut_index.index]
    embedding_tensor = embedding_tensor[one_mut_index.index]
    
mutations_data = {
    'name': protein_path,
    'mutations': mutations['mut_type'].to_list(),
    'prott5': embedding_tensor,
    'coords': coords_tensor,
    'one_hot': one_hot_tensor,
    'delta_g': delta_g_tensor,
    'masks': mask_tensor
}


FileNotFoundError: [Errno 2] No such file or directory: 'Mutation-fineTuning\\mutation_data\\1A0N.csv'

In [ ]:
#print mutation data information
print(f"Protein: {mutations_data['name']}")
print(f"Number of mutations: {len(mutations_data['mutations'])}")
print(f"Protein T5 embeddings shape: {mutations_data['prott5'].shape}")
print(f"Protein coords shape: {mutations_data['coords'].shape}")
print(f"Protein one hot encodings shape: {mutations_data['one_hot'].shape}")
print(f"Protein delta G shape: {mutations_data['delta_g'].shape}")
print(f"Protein masks shape: {mutations_data['masks'].shape}")
